# UT5: Pre-procesamiento de Datos. Limpieza y Transformación Avanzada

con Pandas

Análisis de datos con Python

# UT5: Limpieza y Calidad de Datos

> **Cuaderno de trabajo — UT5: Pre-procesamiento Avanzado**
>
> Este notebook contiene los ejercicios de la unidad. Para la teoría
> completa consulta el libro (PDF).

**Manos a la Obra**

### Diagnóstico de calidad en datos de salud

### Laboratorio de Pruebas: Dataset de Salud Pública

**Objetivo:** Realizar una auditoría técnica completa para identificar
fallos de integridad en el dataset `df_salud`.

**Contexto profesional:** Has recibido el volcado de la base de datos de
ingresos hospitalarios. Antes de cualquier análisis médico, debes
certificar la calidad de los datos para evitar diagnósticos erróneos
basados en información sucia.

**Criterio de éxito:**

-   Informe de nulos por columna generado.
-   Número total de duplicados identificado.
-   Confirmación de si existe un patrón en la falta de datos de peso.

**Tiempo estimado:** 10 minutos

In [4]:
import pandas as pd
import numpy as np

# Generación del dataset df_salud para ejercicios
np.random.seed(42)
n_records = 1200

df_salud = pd.DataFrame({
    'paciente_id': range(1, n_records + 1),
    'fecha_ingreso': pd.to_datetime(np.random.choice(pd.date_range('2023-01-01', '2023-12-31'), n_records)),
    'edad': np.random.randint(18, 95, n_records),
    'peso_kg': np.random.uniform(50, 120, n_records),
    'altura_cm': np.random.uniform(150, 200, n_records),
    'presion_sistolica': np.random.normal(120, 15, n_records),
    'diagnostico': np.random.choice(['Gripe', 'Fractura', 'Control', 'Infección', 'Alergia'], n_records),
    'hospital': np.random.choice(['General', 'Norte', 'Clinico', 'Materno'], n_records),
    'coste_tratamiento': np.random.uniform(100, 5000, n_records)
})

# Introducir problemas intencionadamente
# 1. Nulos MAR (Peso falta más en ancianos > 80 años)
df_salud.loc[df_salud['edad'] > 80, 'peso_kg'] = np.nan
# 2. Outliers en presión
df_salud.loc[np.random.choice(n_records, 10), 'presion_sistolica'] = [300, 310, 280, 290, 320, 10, 5, 8, 12, 15]
# 3. Duplicados
df_salud = pd.concat([df_salud, df_salud.iloc[:15]], ignore_index=True)

**Instrucciones (Guiado):**

1.  Usa `.info()` y `.describe()` para obtener una visión general de
    `df_salud`.
2.  Calcula el porcentaje exacto de valores nulos por columna. ¿Cuál es
    la columna más problemática?
3.  Identifica y cuenta cuántos registros están duplicados exactamente.
4.  **Análisis:** ¿Notas alguna relación entre la edad y la falta de
    datos en la columna `peso_kg`? (Pista: compara la edad media de los
    registros con peso vs. sin peso).

**Manos a la Obra**

### imputación inteligente

**Objetivo:** Aplicar técnicas de imputación que respeten la estructura
de los subgrupos poblacionales para evitar distorsiones estadísticas.

**Contexto profesional:** Como has descubierto, los datos de `peso_kg`
en el dataset `df_salud` faltan de forma sistemática en pacientes
mayores de 80 años. Si imputas con la media de todo el hospital,
subestimarás el peso de los jóvenes y sobreestimarás el de los ancianos.

**Instrucciones (Semi-guiado):**

1.  Crea una copia de seguridad de tu dataset.
2.  Identifica la mediana de peso para cada grupo de edad (puedes crear
    rangos de edad o agrupar por décadas si lo prefieres, pero para este
    ejercicio agrupa por la columna `hospital`).
3.  **Imputación por subgrupos:** Rellena los nulos de `peso_kg`
    utilizando la mediana del hospital al que pertenece el registro.
4.  **Validación:** Compara el peso medio por hospital antes y después
    de la imputación. ¿Se ha mantenido la jerarquía de pesos entre
    hospitales o se ha aplanado la diferencia?

**Pistas:**

-   No copies el código de la teoría directamente. Piensa en el flujo:
    Agrupar por hospital → Seleccionar columna peso → Transformar
    rellenando nulos con la mediana.

**Criterio de éxito:** Los valores faltantes de `peso_kg` se han
rellenado con la mediana de su hospital respectivo, preservando la
variabilidad entre centros. Medianas de referencia: General → 83.1 kg,
Norte → 83.2 kg, Clínico → 88.4 kg, Materno → 87.4 kg. Tras la
imputación, `peso_kg` no debe tener ningún nulo.

La técnica de `.transform()` es avanzada pero muy potente para
imputación profesional. Tienes un recordatorio en el **Apéndice C.4**.

**Tiempo estimado:** 15 minutos

In [12]:
# Escribe tu codigo aqui


**Manos a la Obra**

### detectando *outliers*

**Objetivo:** Implementar y comparar los dos métodos estándar de
detección de *outliers* (IQR vs. *Z-Score*) para identificar lecturas de
presión arterial físicamente imposibles o críticas.

**Contexto profesional:** En el dataset `df_salud`, la columna
`presion_sistolica` contiene valores extremos. Debes determinar cuál de
los dos métodos es más adecuado para este tipo de datos médicos.

**Instrucciones (Semi-guiado):**

1.  **Método IQR:** Calcula $Q_1, Q_3$ e $IQR$ para la presión
    sistolica. Identifica cuántos registros quedan fuera del rango
    $[Q_1 - 1.5 \times IQR, Q_3 + 1.5 \times IQR]$.
2.  **Método Z-Score:** Calcula el *z-score* para la misma columna e
    identifica cuántos registros tienen un valor absoluto superior a 3
    ($|z| > 3$).
3.  **Comparativa:** Muestra cuántos *outliers* ha detectado cada
    método. ¿Coinciden en el número de anomalías?
4.  **Juicio clínico:** Observa los valores detectados como *outliers*
    inferiores (cercanos a 0). ¿Deberían ser imputados o eliminados?
    Justifica tu decisión basándote en la viabilidad biológica.

**Pistas:**

El método IQR es el estándar en la industria para distribuciones no
normales. Tienes la fórmula y el código en el **Apéndice C.4**.

**Criterio de éxito:** El código debe mostrar una comparativa numérica
de ambos métodos. Debes ser capaz de explicar por qué un método ha
detectado más (o menos) registros que el otro.

**Tiempo estimado:** 20 minutos

In [17]:
# Escribe tu codigo aqui


**Manos a la Obra**

### *feature engineering* temporal

**Objetivo:** Transformar columnas de fecha en variables numéricas y
categóricas útiles para detectar estacionalidad en la ocupación
hospitalaria.

**Contexto profesional:** La gerencia del hospital sospecha que los
ingresos por determinadas patologías aumentan durante los fines de
semana. Necesitas descomponer la `fecha_ingreso` para confirmar esta
tendencia.

**Instrucciones (Semi-guiado):**

1.  Verifica que la columna `fecha_ingreso` tiene el tipo de dato
    correcto (datetime).
2.  **Descomposición:** Crea tres nuevas columnas: `mes_ingreso`,
    `dia_semana` (0-6) y `es_fin_de_semana` (booleano).
3.  **Análisis estacional:** Calcula el número total de ingresos por
    cada mes del año.
4.  **Validación de hipótesis:** ¿Es el coste medio del tratamiento
    mayor los fines de semana que los días laborables? Usa una
    agrupación para responder.

**Pistas:**

El manejo de fechas con `.dt` de Pandas es esencial. Tienes los
atributos más comunes en el **Apéndice C.4**.

**Criterio de éxito:** El DataFrame debe contener las 3 nuevas columnas
calculadas. Debes presentar una pequeña tabla comparativa de costes
(Media Laborable vs. Media Fin de Semana).

**Tiempo estimado:** 15 minutos

In [26]:
# Escribe tu codigo aqui


**Manos a la Obra**

### codificación de categóricas

**Objetivo:** Transformar variables cualitativas en formatos numéricos
procesables por algoritmos de Machine Learning, gestionando el impacto
en la dimensionalidad.

**Contexto profesional:** Debes preparar el campo `diagnostico` para un
modelo predictivo. Sin embargo, en un escenario real, los diagnósticos
pueden ser cientos. Debes elegir la técnica de codificación más
eficiente.

**Tu Misión (Autónomo):**

1.  Aplica la técnica de **One-Hot Encoding** a la columna `hospital`.
    Asegúrate de evitar la trampa de la multicolinealidad eliminando la
    primera columna generada (`drop_first=True`).
2.  **Reto de Cardinalidad:** Observa la columna `diagnostico`. Si
    tuviera 50 valores diferentes en lugar de 5, ¿qué pasaría con el
    ancho de tu DataFrame si usaras One-Hot Encoding? Propón (en un
    comentario) una alternativa para manejar categorías con alta
    cardinalidad.
3.  Ejecuta la transformación y muestra las primeras 3 filas del
    DataFrame resultante.

**Criterio de éxito:** El DataFrame final debe haber sustituido la
columna `hospital` por columnas binarias (ej: `hospital_Norte`,
`hospital_Clinico`, etc.).

**Tiempo estimado:** 10 minutos

`pd.get_dummies()` es la forma más rápida de aplicar One-Hot Encoding en
Pandas. Tienes un ejemplo en el **Apéndice C.4**.

In [29]:
# Introduce aquí el código

**Manos a la Obra**

### *pipeline* completo

**Objetivo:** Construir una función de pre-procesamiento profesional que
no solo transforme los datos, sino que audite el proceso mediante
*logging* y valide la integridad de las entradas.

**Contexto profesional:** El equipo de ingeniería de datos requiere que
tu pipeline sea “auditable”. Necesitan un registro de cuántos registros
se eliminan en cada paso y una validación que detenga el proceso si el
dataset de entrada no tiene las columnas esperadas.

**Tu Misión (Autónomo):**

1.  **Validación de entrada:** La función `hospital_data_pipeline(df)`
    debe verificar primero si existen las columnas críticas
    (`paciente_id`, `edad`, `presion_sistolica`). Si falta alguna, debe
    lanzar una excepción o un mensaje de error claro.
2.  **Auditoría (Logging):** En lugar de simples `print()`, utiliza la
    librería `logging` de Python para registrar el inicio del proceso,
    el número de duplicados eliminados y el valor medio del coste antes
    y después del escalado.
3.  **Transformaciones Secuenciales:**
    -   Elimina duplicados.
    -   Impute `peso_kg` usando la mediana por hospital (usa
        `.transform()`).
    -   Aplica un *capping* (recorte) a `presion_sistolica` para que
        ningún valor sea inferior a 60 ni superior a 200.
    -   Crea la columna `mes_ingreso` a partir de la fecha.
    -   Estandarice el `coste_tratamiento` usando `StandardScaler` de
        Scikit-Learn.
4.  **Prueba de robustez:** Ejecuta tu función con `df_salud`. Luego,
    intenta ejecutarla con un DataFrame vacío o al que le falte una
    columna para verificar que tu validación funciona.

**Criterio de éxito:** Entrega de un código que produzca un dataset
limpio y un archivo `.log` (o salida de log en consola) que documente
cuantitativamente cada paso. El proceso debe fallar de forma con
controlado ante datos corruptos.

**Tiempo estimado:** 20 minutos

Un pipeline robusto es la marca de un profesional. Tienes una guía de
los pasos habituales en el **Apéndice C.4**.

In [36]:
# Introduce aquí el código